### Setup and Imports

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
import config
from src import data_ingestion
from src.symbols import Symbols
from src.portfolio1 import Portfolio
from src.benchmark import Benchmark

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Load Transaction Data

In [ ]:
# Create a list of all your transaction log files
log_files = [
    config.TRANS_LOG_DIR / "us_mkt_transactions.csv",
    config.TRANS_LOG_DIR / "exus_mkt_transactions.csv",
]

# Load and merge all found logs into a single DataFrame
master_log = data_ingestion.create_master_log(log_files).set_index("Date")
display(master_log)

### Initialize and Assess Symbols

In [ ]:
# 1. Initialize the Symbol manager
symbol_manager = Symbols(master_log)

# 2. Assess all symbols (checks cache and yfinance, creates templates for missing symbols)
symbol_manager.assess()

# 3. Display the results for user verification
print("✅ The following symbols were automatically identified:")
found_df = symbol_manager.get_found()
display(found_df)

print("❌ The following symbols were NOT found:")
missing_list = symbol_manager.get_missing()
print(missing_list)

### User Correction Step

Reviews the output from the cell above and lists any symbols that were incorrectly identified by the data provider (e.g. `yfinance`).

In [ ]:
# User reviews the table above and populates this list
incorrectly_identified_symbols = ["AMR", "ASM"]

# Mark these symbols as user-provided, which also updates the metadata template
symbol_manager.mark_as_manual(incorrectly_identified_symbols)

### Manual Task

1) Open the file: `data/manual-source/metadata.json`.
2) Fill in the `null` values for all the symbols listed.
3) For each of those same symbols, add a price history CSV file to the `data/manual-source/` directory.

### Reload Metadata and Get Final DataFrame

In [ ]:
# Reload the user-edited file from disk
symbol_manager.reload_user_metadata()

# Get the final, combined DataFrame of all symbol metadata
symbol_df = symbol_manager.get_unified_df()

print("--- Final Unified Symbol Metadata ---")
display(symbol_df)

### Initialize and Run Portfolio Analysis

In [ ]:
# Get the project date range
start_date, end_date, date_range, last_market_day = config.project_dates(
    master_log.index
)

# 1. Initialize the Portfolio analysis engine
portfolio = Portfolio(master_log, symbol_manager, date_range, last_market_day)

# 2. Run all calculations (fetching prices, calculating holdings and value)
portfolio.calculate_holdings_and_value()

portfolio_value = portfolio.get_total_value_history()

### Display Portfolio Results

In [ ]:
portfolio_value

In [ ]:
portfolio.get_current_holdings().round(2).sort_values(
    "Market Value (USD)", ascending=False
)

In [ ]:
portfolio.get_return_summary().round(2)

In [ ]:
total_returns = (
    portfolio.holdings["unrealized_gains"]
    + portfolio.holdings["realized_gains"]
    + portfolio.holdings["income"]
)

In [ ]:
fig_holdings = go.Figure()

for symbol in total_returns.columns:
    fig_holdings.add_trace(
        go.Scatter(
            x=total_returns.index,
            y=total_returns[symbol],
            mode="lines",
            name=symbol,
            line=dict(width=1),
        )
    )

fig_holdings.update_layout(
    title_text="<b>Individual Holding Performance Over Time</b>",
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title=f"Total Returns ({config.BASE_CURRENCY})",
)

fig_holdings.show()

- Inspect this TQQQ issue

### Initialize and Run the Benchmark Simulation

In [ ]:
# Create an instance of the Benchmark class
benchmark_simulation = Benchmark(master_log, date_range, last_market_day)

# Run benchmark simulation
benchmark_simulation.run_simulation()

### Display Benchmark Result

In [ ]:
benchmark_results = benchmark_simulation.get_results()
benchmark_results

Ensure that if base currency is AED or INR and the benchmark prices are in non-base currency, that all benchmark prices are converted to base currency.

### Visualize Portfolio & Benchmark Performance

In [ ]:
# --- 1. Prepare data for the chart ---
# Resample the daily net deposits to get the total for each month
monthly_deposits = benchmark_results["NetDeposit"].resample("ME").sum()

# --- 2. Create the Plotly Figure ---
fig = make_subplots(specs=[[{"secondary_y": True}]])

# --- 3. Add Traces to the Figure ---

# Trace 1: Personal Portfolio Value
fig.add_trace(
    go.Scatter(
        x=portfolio_value.index,
        y=portfolio_value,
        mode="lines",
        name="Personal Portfolio",
        line=dict(color="green", width=2),
    ),
    secondary_y=False,
)

# Trace 2: Benchmark Value
fig.add_trace(
    go.Scatter(
        x=benchmark_results.index,
        y=benchmark_results["TotalValue"],
        mode="lines",
        name=f"{config.BENCHMARK_INDEX} Benchmark",
        line=dict(color="red", width=2),
    ),
    secondary_y=False,
)

# Trace 3: Cumulative Net Deposits
fig.add_trace(
    go.Scatter(
        x=benchmark_results.index,
        y=benchmark_results["NetDeposit"].cumsum(),
        mode="lines",
        name="Cumulative Net Deposits",
        line=dict(color="darkgrey", width=1, dash="dash"),
    ),
    secondary_y=False,
)

# Trace 4: Monthly Deposits/Withdrawals (on secondary y-axis)
fig.add_trace(
    go.Bar(
        x=monthly_deposits.index,
        y=monthly_deposits,
        name="Deposits / Withdrawals",
        marker_color="royalblue",
        opacity=0.4,
    ),
    secondary_y=True,
)

# --- 5. Update Layout and Styling ---
fig.update_layout(
    title_text=f"Portfolio Performance vs. {config.BENCHMARK_INDEX} Benchmark",
    template="plotly_white",
    barmode="relative",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis=dict(
        title="Date",
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    ),
)

# Update Y-Axes Titles
fig.update_yaxes(
    title_text=f"<b>Portfolio Value ({config.BASE_CURRENCY})</b>", secondary_y=False
)
fig.update_yaxes(
    title_text="<b>Monthly Cash Flow</b>",
    secondary_y=True,
    showgrid=False,
    layer="below traces",
)

fig.show()

In [ ]:
portfolio_income = portfolio.get_monthly_income()
benchmark_income = benchmark_simulation.get_monthly_income()

fig_income_comp = go.Figure()

fig_income_comp.add_trace(
    go.Bar(
        x=portfolio_income.index,
        y=portfolio_income,
        name="Portfolio Income",
        marker_color="mediumseagreen",
    )
)

fig_income_comp.add_trace(
    go.Bar(
        x=benchmark_income.index,
        y=benchmark_income,
        name=f"{config.BENCHMARK_INDEX} Benchmark Income",
        marker_color="grey",
    )
)

fig_income_comp.update_layout(
    title_text="<b>Monthly Income Comparison</b>",
    template="plotly_white",
    barmode="group",
    xaxis_title="Date",
    yaxis_title=f"Net Income ({config.BASE_CURRENCY})",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig_income_comp.show()